In [1]:
import numpy as np
import pandas as pd
from simple.renko import Renko
from simple.jurik import JTPO, JRSX
from simple.chart import interactFigure
from simple.pretty import pmap
from simple.hurst import HurstDSOD
from simple.mann_kendall import original_series

In [2]:
C = np.load('data/ETHUSDT.2022-05-01.debounce.npz')['ETHUSDT'].view(np.recarray)
len(C), C

(6824488,
 rec.array([('2022-05-01T00:00:06.912000',       0, 2726.665, 6300000, -5.5435e+04,  922., -5.6357e+04),
            ('2022-05-01T00:00:10.695999',       1, 2726.635, 3783999, -3.3200e+03, 7235., -1.0555e+04),
            ('2022-05-01T00:00:10.724999',       2, 2726.565,       0, -2.8000e+01,    0., -2.8000e+01),
            ...,
            ('2022-05-26T23:59:59.691999', 6824485, 1792.385,       0, -1.0000e+03,    0., -1.0000e+03),
            ('2022-05-26T23:59:59.923000', 6824486, 1792.325,  231001, -1.0575e+04,   83., -1.0658e+04),
            ('2022-05-26T23:59:59.955000', 6824487, 1792.285,       0, -4.7000e+01,    0., -4.7000e+01)],
           dtype=[('DateTime', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8')]))

In [3]:
count = 30

In [4]:
def create(step):
    rnk = Renko(step, C['Price'])
    rnk.create_renko()

    X = pd.DataFrame(rnk.bricks)
    X['mid'] = (X.close + X.open) / 2
    return X.mid.values

In [5]:
R = pmap(create, range(1, count))

  0%|          | 0/29 [00:00<?, ?it/s]

In [6]:
def model(step: int = (0, count-1, 1), period: int = (4, 100, 2)):
    renko = R[step]
    TPO = JTPO(renko, period)
    RSX = JRSX(renko, period)
    Hurst_DSOD = HurstDSOD(renko, period)
    Mann = original_series(renko, period)
    return locals()

In [7]:
interactFigure(model,
    renko={'color': 'gray', 'opacity': 0.4, 'mode': 'lines'}, 
    TPO={'color': 'orange', 'row': 2},
    Mann={'color': 'magenta', 'row': 2},
    RSX={'color': 'red', 'row': 3, 'secondary_y': True},
    Hurst_DSOD={'color': 'blue', 'row': 3},
)